In [1]:
from pathlib import Path
import pandas as pd
from torchmetrics import F1Score, Precision, Recall, Accuracy
import torch
from image_modalities_classifier.models.predict import ModalityPredictor, RunConfig, SingleModalityPredictor
from os import cpu_count


artifacts = Path("/media/cumulus/curation_data/modality_classifiers_production/models/cord19/")
data_path = Path("/media/cumulus/curation_data/modality_classifiers_production/data/cord19")

classifiers = {
    "classifier": "higher-modality",
    "classname": "",
    "path": artifacts / "higher-modality/efficientnet-b1_higher-modality_0.pt",
    "children": [
        {
            "classifier": "experimental",
            "classname": "exp",
            "path": artifacts / "experimental/efficientnet-b1_experimental_0.pt",
            "children": [
                {
                    "classifier": "gel",
                    "classname": "exp.gel",
                    "path": artifacts / "gel/efficientnet-b1_gel_0.pt",
                    "children": []
                }
            ],
        },
        {
            "classifier": "graphics",
            "classname": "gra",
            "path": artifacts / "graphics/efficientnet-b1_graphics_0.pt",
            "children": [],
        },
        {
            "classifier": "microscopy",
            "classname": "mic",
            "path": artifacts / "microscopy/efficientnet-b0_microscopy_0.pt",
            "children": [
                {
                    "classifier": "electron",
                    "classname": "mic.ele",
                    "path": artifacts / "electron/efficientnet-b1_electron_0.pt",
                    "children": [],
                },
            ],
        },
        {
            "classifier": "molecular",
            "classname": "mol",
            "path": artifacts / "molecular" / "efficientnet-b1_molecular_3.pt",
            "children": [],
        },
        {
            "classifier": "radiology",
            "classname": "rad",
            "path": artifacts / "radiology" / "efficientnet-b0_radiology_3.pt",
            "children": [],
        },
        {
            "classifier": "photography",
            "classname": "pho",
            "path": artifacts / "photography/resnet34_photography_0.pt",
            "children": []
        }
    ],
}



model_path = data_path / "cord19_molecular_v3.parquet"
df = pd.read_parquet(model_path)
# df = df[:10]

In [43]:


# config = RunConfig(32, cpu_count(), "cuda:0")

# resnet_34 = "/media/cumulus/curation_data/modality_classifiers/models/cord19/molecular/resnet34_molecular_3.pt"
# efficient = str(artifacts / "molecular/efficientnet-b1_molecular_3.pt")

# predictor = SingleModalityPredictor(efficient, config)
# predictor = ModalityPredictor(classifiers, config)

/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [34]:
base_img_path = "/media/cumulus/curation_data/"
predictions = predictor.predict(train_df, base_img_path)
ground_truth = train_df.label.values

100%|██████████| 34/34 [00:02<00:00, 11.83it/s]


In [52]:
from image_modalities_classifier.models.modality_module import ModalityModule, EfficientNet, IResnet

# ckp_resnet = "/media/cumulus/curation_data/modality_classifiers/models/cord19/microscopy/resnet18_microscopy_2.pt"
ckp_resnet = "/media/cumulus/curation_data/vil-al-interface/models/cord19/radiology/radiology_1.pt"
# ckp_resnet = '/home/jtt/Documents/resnet18_radiology_pickle_3.pt'
ckp = str(artifacts / "molecular/efficientnet-b1_molecular_6-epoch=7-val_loss=0.072.pt")

# model_pl = ModalityModule.load_from_checkpoint(ckp_resnet, strict=True)
# print(type(model_pl))
checkpoint = torch.load(ckp_resnet)
print(checkpoint.keys())

print(checkpoint["hyper_parameters"])


# model = EfficientNet(name="efficientnet-b1", num_classes=4)
resnet_model = IResnet(name="resnet18", num_classes=checkpoint["hyper_parameters"]["num_classes"], pretrained=True, fine_tuned_from="whole")

print(checkpoint["state_dict"].keys())
state_dict = {}
for key in checkpoint["state_dict"].keys():
    if key != 'loss.weight':
      # new_key = f"model.{key}"
      new_key = key
      state_dict[new_key] = checkpoint["state_dict"][key]

# for key in checkpoint["state_dict"].keys():    
#     new_key = f"model.{key}"
#     state_dict[new_key] = checkpoint["state_dict"][key]

# checkpoint["state_dict"] = state_dict

# import pickle

# with open('/home/jtt/Documents/resnet18_radiology_pickle_3.pt', 'wb') as handle:
#     pickle.dump(checkpoint, handle, protocol=pickle.HIGHEST_PROTOCOL)

resnet_model.load_state_dict(state_dict)
# resnet_model


dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])
{'name': 'resnet18', 'num_classes': 4, 'pretrained': True, 'fine_tuned_from': 'whole', 'lr': 0.0001, 'class_weights': array([5.08064516, 0.4600779 , 3.28125   , 0.75479233]), 'metric_monitor': 'val_avg_loss', 'mode_scheduler': 'min', 'mean_dataset': tensor([0.3379, 0.3450, 0.3462]), 'std_dataset': tensor([0.2848, 0.2879, 0.2885])}
odict_keys(['model.conv1.weight', 'model.bn1.weight', 'model.bn1.bias', 'model.bn1.running_mean', 'model.bn1.running_var', 'model.bn1.num_batches_tracked', 'model.layer1.0.conv1.weight', 'model.layer1.0.bn1.weight', 'model.layer1.0.bn1.bias', 'model.layer1.0.bn1.running_mean', 'model.layer1.0.bn1.running_var', 'model.layer1.0.bn1.num_batches_tracked', 'model.layer1.0.conv2.weight', 'model.layer1.0.bn2.weight', 'model.layer1.0.bn2.bias', 'model.layer1.0.bn2.running_mean', 'model.layer1.0.bn2.runnin

<All keys matched successfully>

In [54]:
from typing import List

base_img_path = "/media/cumulus/curation_data/"
# all_data = data_path / "cord19_radiology_v3.parquet"
all_data = "/media/cumulus/curation_data/modality_classifiers/data/cord19/cord19_radiology_v1.parquet"
# efficient = str(artifacts / "molecular/efficientnet-b1_molecular_3.pt")
# efficient = str(artifacts / "molecular/efficientnet-b1_molecular_10-epoch=7-val_loss=0.072.pt")
efficient = str(artifacts / "radiology/efficientnet-b0_radiology_3.pt")
# resnet = "/media/cumulus/curation_data/modality_classifiers/models/cord19/microscopy/resnet18_microscopy_2.pt"
resnet = "/media/cumulus/curation_data/vil-al-interface/models/cord19/radiology/radiology_1.pt"
# efficient = str(artifacts / "molecular/last.pt")

# all_data = data_path / "cord19_graphics_v4.parquet"
# efficient = str(artifacts / "graphics/efficientnet-b1_graphics_4.pt")

df = pd.read_parquet(all_data)
train_df = df.loc[df.split_set == 'TRAIN']
val_df = df.loc[df.split_set=='VAL']
test_df = df.loc[df.split_set=='TEST']

config = RunConfig(32, cpu_count(), "cuda:0")
predictor = SingleModalityPredictor(efficient, config)
# predictor.model = resnet_model

def test(data: pd.DataFrame, predictor: SingleModalityPredictor):
    y_pred = predictor.predict(data, base_img_path)
    y_true = data.label.values
    encoded_y_pred = torch.Tensor(predictor.decoder.transform(y_pred))
    encoded_y_true = torch.Tensor(predictor.decoder.transform(y_true))    
    return calc_stats(encoded_y_true, encoded_y_pred, len(predictor.classes)), y_pred

def calc_stats(y_true: torch.Tensor, y_pred: torch.Tensor, num_classes: int):
    params = {"task": "multiclass", "num_classes": num_classes}

    macro_params = {**params, "average": "macro"}
    f1_macro_metric = F1Score(**macro_params)
    micro_params = {**params, "average": "micro"}
    f1_micro_metric = F1Score(**micro_params)
    weighted_params = {**params, "average": "weighted"}
    f1_wei_metric = F1Score(**weighted_params)
    accuracy = Accuracy(**params)

    return {
        "f1_macro": f1_macro_metric(y_true, y_pred),
        "f1_micro": f1_micro_metric(y_true, y_pred),
        "f1_weighted": f1_wei_metric(y_true, y_pred),
        "accuracy": accuracy(y_true, y_pred)
    }

train_stats, _ = test(train_df, predictor)
val_stats, _ = test(val_df, predictor)
test_stats, y_test_pred = test(test_df, predictor)

print("stats")
print(train_stats)
print(val_stats)
print(test_stats)


# y_preds2_, y_probs2 = predictor.predict_with_probs(test_df, base_img_path)

/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 19/19 [00:03<00:00,  4.76it/s]

stats
{'f1_macro': tensor(0.8794), 'f1_micro': tensor(0.9306), 'f1_weighted': tensor(0.9268), 'accuracy': tensor(0.9306)}
{'f1_macro': tensor(0.8539), 'f1_micro': tensor(0.9110), 'f1_weighted': tensor(0.9080), 'accuracy': tensor(0.9110)}
{'f1_macro': tensor(0.8816), 'f1_micro': tensor(0.9288), 'f1_weighted': tensor(0.9265), 'accuracy': tensor(0.9288)}


In [9]:
from sklearn.calibration import LabelEncoder
from tqdm import tqdm
from numpy import hstack

from image_modalities_classifier.dataset.image_dataset import EvalImageDataset
from image_modalities_classifier.models.modality_module import ModalityModule, EfficientNet, IResnet
from torch.utils.data import DataLoader

from image_modalities_classifier.dataset.transforms import ModalityTransforms
base_img_path = "/media/cumulus/curation_data/"

classifiers = {
    # "higher-modality":"/media/cumulus/curation_data/vil-al-interface/models/cord19/higher-modality/higher-modality_1.pt",
    # "experimental":"/media/cumulus/curation_data/vil-al-interface/models/cord19/experimental/experimental_1.pt",    
    "graphics":"/media/cumulus/curation_data/vil-al-interface/models/cord19/graphics/graphics_1.pt",
    # "microscopy":"/media/cumulus/curation_data/vil-al-interface/models/cord19/microscopy/microscopy_1.pt",
    # "molecular":"/media/cumulus/curation_data/vil-al-interface/models/cord19/molecular/molecular_1.pt",
    # "radiology": "/media/cumulus/curation_data/vil-al-interface/models/cord19/radiology/radiology_1.pt",
}

classes = {
    "higher-modality": ["exp","gra","mic","mol","oth","pho","rad"],
    "experimental": ["exp.gel","exp.pla"],    
    "graphics": ["gra.3dr","gra.flow","gra.his","gra.lin","gra.oth","gra.sca","gra.sig"],
    "microscopy": ["mic.ele","mic.flu","mic.lig"],
    "molecular": ["mol.3ds", "mol.che", "mol.dna", "mol.pro"],
    "radiology": ["rad.ang", "rad.cmp", "rad.uls", "rad.xra"],
}

data_path = Path("/media/cumulus/curation_data/vil-al-interface/files/cord19/")

def load_model(p):
    checkpoint = torch.load(p)
    resnet_model = IResnet(name="resnet18", num_classes=checkpoint["hyper_parameters"]["num_classes"], pretrained=True, fine_tuned_from="whole")
    state_dict = {}    
    for key in checkpoint["state_dict"].keys():
        if key != 'loss.weight':
            new_key = key
            state_dict[new_key] = checkpoint["state_dict"][key]
    resnet_model.load_state_dict(state_dict)
    return resnet_model, checkpoint["hyper_parameters"]["mean_dataset"], checkpoint["hyper_parameters"]["std_dataset"],checkpoint["hyper_parameters"]["num_classes"]

def load_dataloader(data, mean, std):
    transforms_manager = ModalityTransforms("resnet18", mean, std)
    transforms = transforms_manager.test_transforms()
    dataset = EvalImageDataset(data, base_img_path, transforms, path_col="img_path")
    loader = DataLoader(
        dataset=dataset,
        batch_size=32,
        shuffle=False,
        num_workers=4,
        drop_last=False,
    )
    return loader


for key in classifiers.keys():
    filename = f"cord19_{key}_v1.parquet"
    df = pd.read_parquet(data_path / filename)
    print(df.label.unique())
    model, mean, std, num_classes = load_model(classifiers[key])

    decoder = LabelEncoder()
    decoder.fit(classes[key])

    print(key)
    for split in ["VAL", "TEST"]:
        predictions = []
        df_split = df.loc[df.split_set==split]

        model = model.eval().to("cuda:0")
        loader = load_dataloader(df_split, mean, std)

        with torch.no_grad():
            for batch_imgs in tqdm(loader):
                data = batch_imgs.to("cuda:0")
                batch_outputs = model(data)
                batch_predictions = batch_outputs.argmax(dim=-1).cpu()
                predictions.append(batch_predictions)
            prediction_stack = hstack(predictions)

        y_true = df_split.label.values
        encoded_y_true = torch.Tensor(decoder.transform(y_true))

        params = {"task": "multiclass", "num_classes": num_classes}
        macro_params = {**params, "average": "macro"}
        f1_macro_metric = F1Score(**macro_params)
        micro_params = {**params, "average": "micro"}
        f1_micro_metric = F1Score(**micro_params)
        weighted_params = {**params, "average": "weighted"}
        f1_wei_metric = F1Score(**weighted_params)
        accuracy = Accuracy(**params)

        y_true = torch.Tensor(encoded_y_true)
        prediction_stack = torch.Tensor(prediction_stack)
        f1_macro=f1_macro_metric(y_true, prediction_stack)
        f1_micro=f1_micro_metric(y_true, prediction_stack)
        f1_weighted=f1_wei_metric(y_true, prediction_stack)
        acc=accuracy(y_true, prediction_stack)
        print(f"{split}, {f1_macro}, {f1_micro}, {f1_weighted}, {acc}")


['gra.his' 'gra.sca' 'gra.oth' 'gra.lin' None]
graphics


  9%|▊         | 92/1078 [00:16<02:58,  5.52it/s]


KeyboardInterrupt: 

f1_macro tensor(0.9200)
f1_micro tensor(0.9508)
f1_weighted tensor(0.9499)
accuracy tensor(0.9508)


In [17]:
for name, param in predictor.model.named_parameters():
    print(name)
    if name == "model.features.1.0.block.0.0.weight":
        print(param)

model.features.0.0.weight
model.features.0.1.weight
model.features.0.1.bias
model.features.1.0.block.0.0.weight
Parameter containing:
tensor([[[[ 2.0166e-02, -1.0751e-01, -5.8210e-03],
          [-1.2274e-01,  3.2433e-01, -8.6749e-02],
          [-7.4220e-02,  5.3751e-01,  6.5358e-02]]],


        [[[-2.6051e-01, -1.0697e-01, -5.1722e-02],
          [ 1.1956e+00, -4.7957e-01, -4.1340e-02],
          [-2.0706e-01, -1.6970e-01, -6.0464e-02]]],


        [[[-3.2519e-01, -3.8404e-01, -2.5227e-01],
          [-1.0502e-01,  1.3681e+00,  1.4668e-01],
          [-1.0174e-01, -1.5479e-03, -8.9848e-02]]],


        [[[ 3.4568e-02, -5.4957e-01,  8.6564e-02],
          [-3.7460e-01,  2.3233e+00, -4.7775e-01],
          [-8.9641e-02, -1.4202e-01, -3.2014e-03]]],


        [[[ 8.5220e-02, -2.6700e-01,  3.9462e-02],
          [-1.1228e-01,  9.0248e-01, -1.5974e-01],
          [-9.0730e-02,  1.6116e-01, -1.5105e-01]]],


        [[[ 1.9589e-02,  1.8996e-01,  2.0447e-02],
          [ 2.0564e-01, -1.733

In [4]:
test_df["pred"] = y_test_pred

/tmp/ipykernel_4168663/3238558131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["pred"] = y_test_pred


In [7]:
test_df.head()

,img,img_path,width,height,label,source,caption,original,split_set,is_gt,pred
247,5e5f2dc6de000f09242e44b7,tinman/p10209262/10209262/4_1/005.jpg,545.00000000000000,253.00000000000000,mol.pro,tinman,Fig. 1A.,mol.pro,TEST,True,mol.pro
1086,1423-0127-17-13-1.jpg,subfigure-classification/2013/train/GCHE/1423-...,600.00000000000000,268.00000000000000,mol.che,clef13,"Chemical structure of BPR1P0034 .,",GCHE,TEST,True,mol.che
1087,1471-2148-9-206-1.jpg,subfigure-classification/2013/train/GCHE/1471-...,600.00000000000000,375.00000000000000,mol.che,clef13,Simplified GA biosynthetic pathway in plants ....,GCHE,TEST,True,mol.dna
1088,1471-2164-8-95-1.jpg,subfigure-classification/2013/train/GCHE/1471-...,600.00000000000000,360.00000000000000,mol.che,clef13,"Chemical structure of citrinin.,",GCHE,TEST,True,mol.che
1089,1471-2199-6-16-1.jpg,subfigure-classification/2013/train/GGEN/1471-...,600.00000000000000,597.00000000000000,mol.dna,clef13,The zinc-finger (ZF) domain is highly conserve...,GGEN,TEST,True,mol.dna


In [46]:
ground_truth[0]
predictor.classes

array(['mol.3ds', 'mol.che', 'mol.dna', 'mol.pro'], dtype='<U7')

In [40]:
from torchmetrics import F1Score, Precision, Recall
import torch

num_classes = len(predictor.classes)
params = {"task": "multiclass", "num_classes": num_classes}

macro_params = {**params, "average": "macro"}
f1_metric = F1Score(**macro_params)

encoded_y_pred = torch.Tensor(predictor.decoder.transform(predictions))
encoded_y_true = torch.Tensor(predictor.decoder.transform(train_df.label.values))

f1 = f1_metric(encoded_y_pred, encoded_y_true)
print(encoded_y_pred)
print(encoded_y_true)
print(f1)


tensor([0., 0., 0.,  ..., 2., 2., 2.])
tensor([0., 0., 0.,  ..., 2., 2., 2.])
tensor(0.8723)


In [28]:
ground_truth = train_df.label.values

from sklearn.metrics import f1_score

# f1_score(ground_truth, predictions, average="macro")
f1_score(ground_truth, predictions, average="macro")

0.87230271411849

In [12]:
ground_truth

array(['rad.cmp', 'rad.cmp', 'rad.cmp', ..., 'rad.xra', 'rad.xra',
       'rad.xra'], dtype=object)

In [13]:
predictions

array(['rad.xra', 'rad.xra', 'rad.xra', ..., 'rad.xra', 'rad.xra',
       'rad.cmp'], dtype='<U7')

In [4]:
df["features"] = list(features)
df.head()

,img,img_path,width,height,label,source,caption,is_gt,original,split_set,features
0,1471-2121-3-29-2.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,541.0,mic.flu,clef13,Fluorescence microscopy of a S2 cell transfect...,True,DMFL,TRAIN,"[-0.004090134, -0.1427809, -0.1732117, -0.1795..."
1,1471-2172-7-1-4.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,600.0,mic.flu,clef13,Distribution of Bu-1 + cells . PALT is seen b...,True,DMFL,TRAIN,"[0.7017001, -0.077621244, -0.13674453, -0.0627..."
2,1471-2180-10-283-1.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,444.0,mic.flu,clef13,Merged image of the phase contrast and fluores...,True,DMFL,TRAIN,"[-0.11191165, -0.17719942, -0.15846144, -0.137..."
3,1471-2180-5-17-9.jpg,subfigure-classification/2013/train/DMFL/1471-...,552.0,548.0,mic.flu,clef13,"An epifluorescence image of Thermus aquaticus .,",True,DMFL,TRAIN,"[1.7194173, -0.08525187, -0.12554392, -0.05411..."
4,1471-2199-11-11-2.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,576.0,mic.flu,clef13,Fluorescence microscopy of tagged-NrdB cells ....,True,DMFL,TRAIN,"[0.18778822, -0.09744128, -0.14088048, -0.1344..."


In [5]:
# restart because the feature extractor is affecting the original model
predictor = SingleModalityPredictor(str(artifacts / "microscopy/efficientnet-b0_microscopy_0.pt"), config)
predictions, probabilities = predictor.predict_with_probs(df, base_img_path)

/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
df["prediction"] = predictions
df["probs"] = probabilities
df.head()

,img,img_path,width,height,label,source,caption,is_gt,original,split_set,features,prediction,probs
0,1471-2121-3-29-2.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,541.0,mic.flu,clef13,Fluorescence microscopy of a S2 cell transfect...,True,DMFL,TRAIN,"[-0.004090134, -0.1427809, -0.1732117, -0.1795...",mic.flu,"[0.001480210805311799, 0.9981943964958191, 0.0..."
1,1471-2172-7-1-4.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,600.0,mic.flu,clef13,Distribution of Bu-1 + cells . PALT is seen b...,True,DMFL,TRAIN,"[0.7017001, -0.077621244, -0.13674453, -0.0627...",mic.flu,"[2.457024311297573e-05, 0.999975323677063, 5.4..."
2,1471-2180-10-283-1.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,444.0,mic.flu,clef13,Merged image of the phase contrast and fluores...,True,DMFL,TRAIN,"[-0.11191165, -0.17719942, -0.15846144, -0.137...",mic.flu,"[0.08060691505670547, 0.873210072517395, 0.046..."
3,1471-2180-5-17-9.jpg,subfigure-classification/2013/train/DMFL/1471-...,552.0,548.0,mic.flu,clef13,"An epifluorescence image of Thermus aquaticus .,",True,DMFL,TRAIN,"[1.7194173, -0.08525187, -0.12554392, -0.05411...",mic.flu,"[2.004333810035064e-09, 1.0, 6.101986537582205..."
4,1471-2199-11-11-2.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,576.0,mic.flu,clef13,Fluorescence microscopy of tagged-NrdB cells ....,True,DMFL,TRAIN,"[0.18778822, -0.09744128, -0.14088048, -0.1344...",mic.flu,"[0.013884089887142181, 0.9833231568336487, 0.0..."


In [17]:
import numpy as np

def cal_margin_sampling(y_pred_prob):
    return np.diff(-np.sort(y_pred_prob)[:, ::-1][:, :2])

def calc_entropy(y_pred_prob):
    return -np.nansum(np.multiply(y_pred_prob, np.log(y_pred_prob)), axis=1)

In [13]:
probs = np.vstack(df.probs)

In [21]:
df["ms"] = cal_margin_sampling(probs)

In [22]:
df["en"] = calc_entropy(probs)

In [24]:
df.head()

,img,img_path,width,height,label,source,caption,is_gt,original,split_set,features,prediction,probs,ms,en
0,1471-2121-3-29-2.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,541.0,mic.flu,clef13,Fluorescence microscopy of a S2 cell transfect...,True,DMFL,TRAIN,"[-0.004090134, -0.1427809, -0.1732117, -0.1795...",mic.flu,"[0.001480210805311799, 0.9981943964958191, 0.0...",0.996714,1.406192e-02
1,1471-2172-7-1-4.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,600.0,mic.flu,clef13,Distribution of Bu-1 + cells . PALT is seen b...,True,DMFL,TRAIN,"[0.7017001, -0.077621244, -0.13674453, -0.0627...",mic.flu,"[2.457024311297573e-05, 0.999975323677063, 5.4...",0.999951,2.863747e-04
2,1471-2180-10-283-1.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,444.0,mic.flu,clef13,Merged image of the phase contrast and fluores...,True,DMFL,TRAIN,"[-0.11191165, -0.17719942, -0.15846144, -0.137...",mic.flu,"[0.08060691505670547, 0.873210072517395, 0.046...",0.792603,4.633903e-01
3,1471-2180-5-17-9.jpg,subfigure-classification/2013/train/DMFL/1471-...,552.0,548.0,mic.flu,clef13,"An epifluorescence image of Thermus aquaticus .,",True,DMFL,TRAIN,"[1.7194173, -0.08525187, -0.12554392, -0.05411...",mic.flu,"[2.004333810035064e-09, 1.0, 6.101986537582205...",1.000000,4.030027e-08
4,1471-2199-11-11-2.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,576.0,mic.flu,clef13,Fluorescence microscopy of tagged-NrdB cells ....,True,DMFL,TRAIN,"[0.18778822, -0.09744128, -0.14088048, -0.1344...",mic.flu,"[0.013884089887142181, 0.9833231568336487, 0.0...",0.969439,9.234295e-02


In [25]:
from sklearn.decomposition import PCA

random_state = 42

pca = PCA(n_components=2, random_state=random_state)
pca.fit(np.vstack(df.features))

PCA(n_components=2, random_state=42)

In [29]:
embeddings_pca  = pca.transform(np.vstack(df.features))
df["x_pca"], df["y_pca"] = embeddings_pca[:,0], embeddings_pca[:, 1]

In [33]:
from sklearn.neighbors import NearestNeighbors

def calc_neighborhood_hit(df, x_col, y_col, n_neighbors=6,column_label='label'):    
    projections = [[i, j] for (i, j) in zip(df[x_col], df[y_col])]
    neigh = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(projections)    
    n_hits = []
    for neighborhood in neigh.kneighbors(projections, n_neighbors + 1, return_distance=False):
        labels  = df.iloc[neighborhood][column_label].values
        targets = [labels[0]] * (len(labels) - 1) 
        n_hit = np.mean(targets == labels[1:])
        n_hits.append(n_hit)
    return n_hits

In [36]:
# from sklearn.preprocessing import LabelEncoder
# predictor.model.hparams

df["hits"] = calc_neighborhood_hit(df, "x_pca", "y_pca", n_neighbors=6)


In [37]:
df.head()

,img,img_path,width,height,label,source,caption,is_gt,original,split_set,features,prediction,probs,ms,en,x_pca,y_pca,hits
0,1471-2121-3-29-2.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,541.0,mic.flu,clef13,Fluorescence microscopy of a S2 cell transfect...,True,DMFL,TRAIN,"[-0.004090134, -0.1427809, -0.1732117, -0.1795...",mic.flu,"[0.001480210805311799, 0.9981943964958191, 0.0...",0.996714,1.406192e-02,4.251132,-1.009367,1.000000
1,1471-2172-7-1-4.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,600.0,mic.flu,clef13,Distribution of Bu-1 + cells . PALT is seen b...,True,DMFL,TRAIN,"[0.7017001, -0.077621244, -0.13674453, -0.0627...",mic.flu,"[2.457024311297573e-05, 0.999975323677063, 5.4...",0.999951,2.863747e-04,13.939093,-2.682397,1.000000
2,1471-2180-10-283-1.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,444.0,mic.flu,clef13,Merged image of the phase contrast and fluores...,True,DMFL,TRAIN,"[-0.11191165, -0.17719942, -0.15846144, -0.137...",mic.flu,"[0.08060691505670547, 0.873210072517395, 0.046...",0.792603,4.633903e-01,-0.888648,-0.570510,0.666667
3,1471-2180-5-17-9.jpg,subfigure-classification/2013/train/DMFL/1471-...,552.0,548.0,mic.flu,clef13,"An epifluorescence image of Thermus aquaticus .,",True,DMFL,TRAIN,"[1.7194173, -0.08525187, -0.12554392, -0.05411...",mic.flu,"[2.004333810035064e-09, 1.0, 6.101986537582205...",1.000000,4.030027e-08,28.581532,-1.882622,1.000000
4,1471-2199-11-11-2.jpg,subfigure-classification/2013/train/DMFL/1471-...,600.0,576.0,mic.flu,clef13,Fluorescence microscopy of tagged-NrdB cells ....,True,DMFL,TRAIN,"[0.18778822, -0.09744128, -0.14088048, -0.1344...",mic.flu,"[0.013884089887142181, 0.9833231568336487, 0.0...",0.969439,9.234295e-02,1.939335,-1.219068,1.000000
